In [2]:
import numpy as np
import scipy as scp

In [10]:
np.random.seed(56)
n_feats = 5
random_matrix = lambda n: np.dot(mat:=np.random.randn(n, n), mat.T)
## initialize multivariate normal dist with normally distributed means and covariance
## drawn from an inverse wishart distribution (conjugate prior for MVN)
norm_means_a = np.random.randn(n_feats)
wishart_cov = scp.stats.invwishart(n_feats, random_matrix(n_feats)).rvs()
## multivariate normal distributions with different means and equal variances
mvn_a = scp.stats.multivariate_normal(mean=norm_means_a, cov=wishart_cov)
mvn_b = scp.stats.multivariate_normal(mean=np.zeros_like(norm_means_a), cov=wishart_cov)
## generate data samples from a multivariate normal
data = np.vstack([mvn_a.rvs(5000), mvn_b.rvs(5000)])
## arbitrary correlations between data variables and classes
betas = np.random.randn(n_feats)
## separate classes
y = data @ betas
labels = np.arange(len(data))<5000
# ## introduce classification noise
# eta = .05
# noisy_labels = np.where(np.random.uniform(size=len(data)) < (1-eta), labels, ~labels)

In [11]:
np.mean(noisy_labels == labels)

0.9479

In [12]:
np.save("random_data_X.npy", data)
np.save("random_data_y", noisy_labels)